# Chisquare test?
- 설명변수가 범주형 변수인 경우 사용하는 통제 기법
- 데이터의 각 범주에 따른 결과 변수의 분포를 설명하거나
- 범주형 변수가 두 개 이상인 경우 두 변수의 상관 여부를 검정하는 것이 목적

-----------------

## 교차분석
- 교차표를 통해 각 셀의 관찰빈도와 기대빈도 간의 차이를 검정하는 기법
- 관찰빈도 : 자료로부터 얻은 빈도분포
- 기대빈도 : 두 변수가 독립일 때 이론적으로 기대할 수 있는 빈도 분포

## 카이제곱 검정
- 카이제곱 검정은 검정 목적에 따라 사용되는 3가지 검정이 있다

1. 적합성 검정
- 각 범주에 따른 데이터의 빈도분포가 이론적으로 기대하는 분포를 따르는가를 검정하는 방법
- ex. 주사위를 굴렸을 때, 각 주사위의 값이 1/6인지를 검정
- scipy.stats.chisquare(f_obs, f_exp=None, ddof=0, axis=0)을 이용
- f_obs는 데이터 관찰빈도 입력, f_exp는 각 범주의 기대 빈도

#### 가설수립
- 귀무가설 : 타이타닉 생존자 중 남, 여의 비율은 각각 50%이다
- 대립가설 : 타이타닉 생존자 중 남, 여의 비율이 각 50%라고 할 수 없다

In [1]:
import pandas as pd

df = pd.read_csv("https://raw.githubusercontent.com/ADPclass/ADP_book_ver01/main/data/titanic.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   survived     891 non-null    int64  
 1   pclass       891 non-null    int64  
 2   sex          891 non-null    object 
 3   age          714 non-null    float64
 4   sibsp        891 non-null    int64  
 5   parch        891 non-null    int64  
 6   fare         891 non-null    float64
 7   embarked     889 non-null    object 
 8   class        891 non-null    object 
 9   adult_male   891 non-null    bool   
 10  embark_town  889 non-null    object 
dtypes: bool(1), float64(2), int64(4), object(4)
memory usage: 70.6+ KB


In [2]:
df_t = df[df['survived']==1]
table = df_t[['sex']].value_counts()
table

sex   
female    233
male      109
Name: count, dtype: int64

In [3]:
from scipy.stats import chisquare

chi = chisquare(table, f_exp=[171, 171])
print('적합성 검정', chi)

적합성 검정 Power_divergenceResult(statistic=44.95906432748538, pvalue=2.011967257447723e-11)


- p-value 값이 유의수준 0.05보다 작으므로 귀무가설을 기각한다.
- 따라서 타이타닉의 생존자 중 남, 여의 비율은 50대 50이라고 할 수 없다

2. 독립성 검정
- 모집단이 두 개의 변수 A, B에 의해 범주화되었을 때 두 변수들 간의 관계가 독립인지 검정하는 방법
- scipy.stats.chi2_contigency((observed, correction=True, lambda_=None))를 사용
- observed 데이터의 관찰빈도, chi2 카이제곱 통계량 값, p p-value, dof 자유도, expected 기대빈도

#### 가설수립
- 귀무가설 : class와 survived는 독립이다
- 대립가설 : class와 survived는 독립이 아니다

In [4]:
table = pd.crosstab(df['class'], df['survived'])
table

survived,0,1
class,,
First,80,136
Second,97,87
Third,372,119


In [5]:
from scipy.stats import chi2_contingency

chi, p, df, expect = chi2_contingency(table)
print(f'statistic : {chi}')
print(f'p-value :  {p}')
print(f'df : {df}')
print(f'expect : {expect}')

statistic : 102.88898875696056
p-value :  4.549251711298793e-23
df : 2
expect : [[133.09090909  82.90909091]
 [113.37373737  70.62626263]
 [302.53535354 188.46464646]]


- 분석결과 p-value의 값의 유의수준 0.05보다 낮으므로 귀무가설을 기각한다
- 좌석등급과 생존은 독립이 아니라고 할 수 있다

3. 동질성 검정
- 모집단이 임의의 변수에 따라 R개의 속성으로 범주화 되었을 때,
- R개의 부분 모집단에서 추출한 표본이 C개의 범주화된 집단의 분포와 서로 동일한지를 검정
- 독립성 검정과 계산법, 검정 방법은 같다

#### 가설수립
- 귀무가설 : class의 분포는 survived에 상관없이 동일하다
- 대립가설 : class의 분포는 survived에 상관없이 동일하지 않다